In [ ]:
## A lambda function that will be triggered on s3 events to process xml log files

In [ ]:
import boto3
import logging
import re
from bs4 import BeautifulSoup
import io, json

def apple_xml_parser(event):
    # checking the input
    try:
        if re.search(r'(.*?)(apple\.xml)', event.key, re.MULTILINE | re.IGNORECASE) == None:
            return

        obj = s3.get_object(Bucket=bucket_name, Key=event.key)
        xml_data = obj['Body']

        xml_soup = BeautifulSoup(xml_data)
        output_json_dict = {}
        
        try:
            output_json_dict['key1'] = xml_soup.find('key1').get_text()
        except:
            output_json_dict['key1'] = ''
            
            
        # we can have multiple data here
        item_soup_list = xml_soup.select('key2 > key3 > key4')
        item_list = []
        for x in item_soup_list:
            item_list.append(x)

        # Some other data extraction ...
        
        output_json_dict_bytes = json.dumps(output_json_dict, indent=4).encode('utf-8')

        output_json_path = (event.key).rsplit('/', 1)[0] + '/generic_info.json'
        #  'nest1/nest2/nest3/generic_info.json'
        s3.put_object(Bucket=bucket_name,
                Key=output_json_path,
                Body=output_json_dict_bytes)

    except Exception as e:
        app.log.error("Error: " + str(e))